@author: SteveEckel

This example covers calculating the example of .

There are three different governing equations that we can use for this example: the `heuristic` equation, the `rateeq` or the `obe`.  In this example, we'll calculate using all three and compare.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pylcp

### Let's start by defining the lasers, magnetic field, and Hamiltonian:

Because we are dealing with a two state system, we keep the Hamiltonian and the geometry relatively simple.  Note that because we are also using the heuristic equation, we want to make sure that the detuning is not on the Hamiltonian, but on the lasers.

In [6]:
# Make a method to return the lasers:
def return_lasers(delta, beta):
    return pylcp.laserBeams([
        {'kvec':np.array([1., 0., 0.]), 'pol':np.array([0., 1., 0.]),
         'pol_coord':'spherical', 'delta':delta, 'beta':beta},
        {'kvec':np.array([-1., 0., 0.]), 'pol':np.array([0., 1., 0.]),
         'pol_coord':'spherical', 'delta':delta, 'beta':beta},
        {'kvec':np.array([1., 0., 0.]), 'pol':np.array([0., 1., 0.]),
         'pol_coord':'spherical', 'delta':-delta, 'beta':beta, 'phase':np.pi/2},
        {'kvec':np.array([-1., 0., 0.]), 'pol':np.array([0., 1., 0.]),
         'pol_coord':'spherical', 'delta':-delta, 'beta':beta, 'phase':np.pi/2},
        ], beam_type=pylcp.infinitePlaneWaveBeam)

# Now define a two level Hamiltonian, connected using pi-light.
def return_hamiltonian(delta):
    Hg = np.array([[0.]])
    He = np.array([[-delta]])
    mu_q = np.zeros((3, 1, 1))
    d_q = np.zeros((3, 1, 1))
    d_q[1, 0, 0] = 1.

    return pylcp.hamiltonian(Hg, He, mu_q, mu_q, d_q)

hamiltonian = return_hamiltonian(0.)

magField = lambda R: np.zeros(R.shape)

### Generate the force profile:

In [ ]:
delta = 39
betas = [2*39**2, 2*43**2, 2*47**2]

hamiltonian = return_hamiltonian(0.)

v = np.arange(-30., 30.1, 0.1)

obe ={}
for beta in betas:
    laserBeams = return_lasers(delta, beta)
    
    obe[beta] = pylcp.obe(laserBeams, magField, hamiltonian)
    obe[beta].generate_force_profile(
        np.zeros((3,) + v.shape),
        [v, np.zeros(v.shape), np.zeros(v.shape)],
        name='molasses', progress_bar=True, deltat_tmax=2*np.pi*10/np.sqrt(beta/2),
        deltat_v=np.inf, itermax=10, rel=1e-4, abs=1e-6
    )